# Construção de um Classificador Binário baseado no algoritmo KNN

## Dependências necessárias

In [ ]:
# In[50]:
from __future__ import absolute_import, division, print_function
import pandas as pd
import nltk  
import numpy as np  
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
import heapq
import re  
import io
import math
import csv
# Helper libraries
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import operator
import PIL
import tqdm
import tqdm.auto
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.preprocessing import StandardScaler  
from IPython.display import display
import os